## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-11-05-00 +0000,wsj,Adyen Shares Hit Two-Year Low After Revenue Mi...,https://www.wsj.com/business/earnings/adyen-sh...
1,2026-02-12-11-00-35 +0000,nyt,Ukrainian Olympian Is Disqualified Over Helmet...,https://www.nytimes.com/2026/02/12/world/olymp...
2,2026-02-12-11-00-00 +0000,nypost,Bergdorf Goodman workers claim majority of cas...,https://nypost.com/2026/02/12/us-news/bergdorf...
3,2026-02-12-11-00-00 +0000,nypost,NYC’s biggest indoor pickleball spot is coming...,https://nypost.com/2026/02/12/business/nycs-bi...
4,2026-02-12-10-56-29 +0000,bbc,Polls close in first election since Gen Z prot...,https://www.bbc.com/news/articles/cy7jdmvn3l0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2499/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
185,trump,39
176,epstein,16
751,canada,16
219,new,16
4,year,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
326,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,139
127,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,120
144,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,98
363,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...,95
343,2026-02-11-14-03-29 +0000,cbc,U.S. House votes against Trump's tariffs on Ca...,https://www.cbc.ca/news/world/trump-tariffs-ca...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
326,139,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
289,80,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
200,58,2026-02-11-21-42-00 +0000,wsj,"At a congressional hearing, Attorney General P...",https://www.wsj.com/politics/policy/pam-bondi-...
109,53,2026-02-12-02-15-54 +0000,nypost,12-year-old girl ID’d as victim in Canada scho...,https://nypost.com/2026/02/11/world-news/girl-...
238,48,2026-02-11-20-02-28 +0000,nypost,Outraged Salesforce employees demand CEO Benio...,https://nypost.com/2026/02/11/business/salesfo...
262,45,2026-02-11-18-56-22 +0000,latimes,Warner Bros. Discovery board faces pressure as...,https://www.latimes.com/entertainment-arts/bus...
140,43,2026-02-11-23-49-00 +0000,wsj,The Republican-led House passed legislation th...,https://www.wsj.com/politics/elections/save-ac...
247,39,2026-02-11-19-35-00 +0000,wsj,LA28 Olympics Organizers Say Wasserman Will Re...,https://www.wsj.com/us-news/casey-wasserman-28...
363,38,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...
161,34,2026-02-11-22-51-00 +0000,wsj,Federal Reserve governor Stephen Miran said he...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
